In [2]:
!pip install -qU flask-ngrok
!pip uninstall fastai -y
!pip install -qU fastai

Uninstalling fastai-1.0.61:
  Successfully uninstalled fastai-1.0.61
     |████████████████████████████████| 194kB 5.7MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 12.8MB 318kB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [8]:
!python -c "import torch; print(torch.__version__)"
!python -c "import fastai; print(fastai.__version__)"

1.7.1
2.3.0


In [3]:
!git clone https://github.com/Vibha27/image-restoration-enhancement.git

Cloning into 'image-restoration-enhancement'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 132 (delta 51), reused 118 (delta 42), pack-reused 0
Receiving objects: 100% (132/132), 5.64 MiB | 6.44 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [4]:
import os
import cv2
import time
import shutil
import numpy as np
import PIL
from PIL import Image
from io import BytesIO
from flask_ngrok import run_with_ngrok
from IPython.display import clear_output
from flask import Flask, render_template, url_for, request, redirect, abort, send_from_directory

In [5]:
#@title Setup
dir = r"/content/image-restoration-enhancement"
for folder in os.listdir(dir):
    src = os.path.join(dir, folder)
    if os.path.isdir(src):
        shutil.copytree(src, f"/content/{folder}")

!rm -r /content/static/uploads/*.jpg /content/static/uploads/*.png /content/static/uploads/*.jpeg
!rm -r /content/image-restoration-enhancement

In [37]:
#@title Model Imports

# # esrgan_tf model
# # # https://drive.google.com/file/d/14twK6Xm_yuL14eTjQbH3aynXDAXPbMLq/view?usp=sharing
!gdown --id 14twK6Xm_yuL14eTjQbH3aynXDAXPbMLq
!unzip -qq esrgan_tf.zip
!mv /content/content/super_model/esrgan /content/esrgan
!rm -r /content/content

# # colorization model
# # # https://drive.google.com/file/d/1f2M62S-5hf8fO6qCaQ1GNYmA6vrrgiwL/view?usp=sharing
!gdown --id 1f2M62S-5hf8fO6qCaQ1GNYmA6vrrgiwL
!unzip -q /content/colorisation.zip 
!mv /content/content/colorisation_model /content/colorisation_model
!rm -r /content/content

# # Dehazing model

!gdown --id 1EGIJADl24Om3Hv7_bJVMPiqU1Dd0kg3b
!mkdir dehazing_model
!mv /content/ots_train_ffa_3_19.pk /content/dehazing_model/ots_train_ffa_3_19.pk

# # Inpainting Model
# https://drive.google.com/file/d/19sd-0YiXULvMQ4Wgeja_yDC4X_vqCrYl/view?usp=sharing
!gdown --id 19sd-0YiXULvMQ4Wgeja_yDC4X_vqCrYl
!unzip -q /content/Image_Inpainting_256_Model.zip

clear_output()

In [38]:
#@title Image Super-resolution: Current Tensorflow
import tensorflow as tf

# MODEL LOADS

super_res_model = tf.keras.models.load_model("/content/esrgan")
super_res_model.compile=False



# SUPER_RES PREPROCESSING TENSORFLOW

def preprocess_image(lr_image):
    if lr_image.shape[-1] == 4:
        lr_image = lr_image[...,:-1]

    lr_image = tf.cast(lr_image, tf.float32)

    lr_image = tf.convert_to_tensor(
        lr_image, dtype=None, dtype_hint=None, name=None
    )
    return tf.expand_dims(lr_image, 0)
    


def super_resolution_module(name, file_path, save_dir):
    og_image = Image.open(file_path).convert("RGB")
    og_image = np.asarray(og_image)
    image = preprocess_image(og_image) 
    fake_image = super_res_model(image)
    fake_image = tf.squeeze(fake_image)
    fake_image = tf.clip_by_value(fake_image, 0, 255)
    fake_image = Image.fromarray(tf.cast(fake_image, tf.uint8).numpy()) 
    
    name = "out_"+name
    save_path = os.path.join(save_dir, name)
    
    fake_image.save(save_path)
    return save_path

# # TEST
# super_resolution_module("000060.png", r"/content/000060.png", r"/content/static/OUTPUTS")
clear_output()

In [39]:
#@title Image Colorization
import torch
import fastai
from torchvision import transforms
from skimage.color import rgb2lab, lab2rgb

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_path = r"/content/colorisation_model/net_G_color.pth"
color_model = torch.load(model_path)
color_model.eval()
color_model.to(device)


def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def lab_to_rgb(L, ab):
    """
    Takes a batch of images
    """

    L = (L + 1.0) * 50.0
    ab = ab * 110.0
    Lab = torch.cat([L, ab], dim=1).permute(0, 2, 3, 1).cpu().numpy()

    rgb_imgs = []
    gray_scales = []

    for img in Lab:
        img_rgb = lab2rgb(img)
        gray_scale = rgb2gray(img_rgb)
        # gray_scale = np.expand_dims(gray_scale)
        rgb_imgs.append(img_rgb)

        gray_scales.append(gray_scale)
    return np.clip(np.concatenate(gray_scales, axis=1), 0, 255), torch.from_numpy(np.clip(np.concatenate(rgb_imgs, axis=1), 0, 255))

def colorisation_module(name, file_path, save_dir):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    og_h, og_w = img.shape[:2]

    img = cv2.resize(img, (256, 256), cv2.INTER_CUBIC)
    img = np.asarray(img)
    img_lab = rgb2lab(img).astype("float32") # Converting RGB to L*a*b
    img_lab = transforms.ToTensor()(img_lab)
    L = img_lab[[0], ...] / 50. - 1. # Between -1 and 1
    L = L.unsqueeze(0)
    L = L.to(device)

    with torch.no_grad():
        output = color_model(L)
    
    L, output = lab_to_rgb(L, output)
    output = output.numpy() 

    output = Image.fromarray(np.uint8(output*255))
    output = output.resize((og_w, og_h), PIL.Image.BICUBIC)

    # print(output.size, type(output), np.max(output), np.min(output))

    save_path = os.path.join(save_dir, f"out_{name}")
    # print(save_path)
    output.save(save_path)
    # print(output.size, type(output))

    return save_path

In [40]:
#@title Image Dehazing 

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.utils import save_image

def default_conv(in_channels, out_channels, kernel_size, bias=True):
    return nn.Conv2d(in_channels, out_channels, kernel_size, padding=(kernel_size//2), bias=bias)
    
    
class PALayer(nn.Module):
    def __init__(self, channel):
        super(PALayer, self).__init__()
        self.pa = nn.Sequential(
                nn.Conv2d(channel, channel // 8, 1, padding=0, bias=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // 8, 1, 1, padding=0, bias=True),
                nn.Sigmoid()
        )
    def forward(self, x):
        y = self.pa(x)
        return x * y

    
class CALayer(nn.Module):
    def __init__(self, channel):
        super(CALayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.ca = nn.Sequential(
                nn.Conv2d(channel, channel // 8, 1, padding=0, bias=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // 8, channel, 1, padding=0, bias=True),
                nn.Sigmoid()
        )

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.ca(y)
        return x * y

    
class Block(nn.Module):
    def __init__(self, conv, dim, kernel_size,):
        super(Block, self).__init__()
        self.conv1 = conv(dim, dim, kernel_size, bias=True)
        self.act1 = nn.ReLU(inplace=True)
        self.conv2 = conv(dim, dim, kernel_size, bias=True)
        self.calayer = CALayer(dim)
        self.palayer = PALayer(dim)

    def forward(self, x):
        res = self.act1(self.conv1(x))
        res = res+x 
        res = self.conv2(res)
        res = self.calayer(res)
        res = self.palayer(res)
        res += x 
        return res

    
class Group(nn.Module):
    def __init__(self, conv, dim, kernel_size, blocks):
        super(Group, self).__init__()
        modules = [Block(conv, dim, kernel_size)  for _ in range(blocks)]
        modules.append(conv(dim, dim, kernel_size))
        self.gp = nn.Sequential(*modules)

    def forward(self, x):
        res = self.gp(x)
        res += x
        return res

    
class FFA(nn.Module):
    def __init__(self,gps,blocks,conv=default_conv):
        super(FFA, self).__init__()
        self.gps = gps
        self.dim = 64
        kernel_size = 3
        pre_process = [conv(3, self.dim, kernel_size)]
        assert self.gps==3
        self.g1 = Group(conv, self.dim, kernel_size,blocks=blocks)
        self.g2 = Group(conv, self.dim, kernel_size,blocks=blocks)
        self.g3 = Group(conv, self.dim, kernel_size,blocks=blocks)
        self.ca = nn.Sequential(*[
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(self.dim*self.gps,self.dim//16,1,padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(self.dim//16, self.dim*self.gps, 1, padding=0, bias=True),
            nn.Sigmoid()
            ])
        self.palayer = PALayer(self.dim)

        post_process = [
            conv(self.dim, self.dim, kernel_size),
            conv(self.dim, 3, kernel_size)]

        self.pre = nn.Sequential(*pre_process)
        self.post = nn.Sequential(*post_process)

    def forward(self, x1):
        x = self.pre(x1)
        res1 = self.g1(x)
        res2 = self.g2(res1)
        res3 = self.g3(res2)
        w = self.ca(torch.cat([res1,res2,res3],dim=1))
        w = w.view(-1,self.gps, self.dim)[:,:,:,None,None]
        out = w[:,0,::] * res1 + w[:,1,::] * res2+w[:,2,::] * res3
        out = self.palayer(out)
        x = self.post(out)
        return x + x1



device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_path = r"/content/dehazing_model/ots_train_ffa_3_19.pk"

ckp = torch.load(model_path, map_location=device)
dehaze_net = FFA(gps=3, blocks=19)
dehaze_net = nn.DataParallel(dehaze_net)
dehaze_net.load_state_dict(ckp['model'])
dehaze_net.eval()


def dehazing_module(name, file_path, save_dir):
    haze = Image.open(file_path)
    haze1 = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.64, 0.6, 0.58],std=[0.14,0.15, 0.152])
    ])(haze)[None,::]

    # haze_no = tfs.ToTensor()(haze)[None,::]

    with torch.no_grad():
        pred = dehaze_net(haze1)

    ts = torch.squeeze(pred.clamp(0,1).cpu())
    print(ts.shape)
    # ts = make_grid(ts, nrow=1, normalize=True)
    save_path = os.path.join(save_dir, f"out_{name}")
    save_image(ts, save_path)
    return save_path

In [41]:
#@title Image Inpainting
from tensorflow import keras
import tensorflow as tf
import numpy as np
from google.colab.patches import cv2_imshow

from tensorflow.keras import backend as K
from tensorflow.keras.layers import InputSpec
from tensorflow.keras.layers import Conv2D


class PConv2D(Conv2D):
    def __init__(self, *args, n_channels=3, mono=False, **kwargs):
        super().__init__(*args, **kwargs)
        self.input_spec = [InputSpec(ndim=4), InputSpec(ndim=4)]

    def build(self, input_shape):        
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
            
        if input_shape[0][channel_axis] is None:
            raise ValueError('The channel dimension of the inputs should be defined. Found `None`.')
            
        self.input_dim = input_shape[0][channel_axis]
        
        # Image kernel
        kernel_shape = self.kernel_size + (self.input_dim, self.filters)
        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='img_kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        # Mask kernel
        self.kernel_mask = K.ones(shape=self.kernel_size + (self.input_dim, self.filters))

        # Calculate padding size to achieve zero-padding
        self.pconv_padding = (
            (int((self.kernel_size[0]-1)/2), int((self.kernel_size[0]-1)/2)), 
            (int((self.kernel_size[0]-1)/2), int((self.kernel_size[0]-1)/2)), 
        )

        # Window size - used for normalization
        self.window_size = self.kernel_size[0] * self.kernel_size[1]
        
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.filters,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.built = True

    def call(self, inputs, mask=None):
        
        # Both image and mask must be supplied
        if type(inputs) is not list or len(inputs) != 2:
            raise Exception('PartialConvolution2D must be called on a list of two tensors [img, mask]. Instead got: ' + str(inputs))

        # Padding done explicitly so that padding becomes part of the masked partial convolution
        images = K.spatial_2d_padding(inputs[0], self.pconv_padding, self.data_format)
        masks = K.spatial_2d_padding(inputs[1], self.pconv_padding, self.data_format)

        # Apply convolutions to mask
        mask_output = K.conv2d(
            masks, self.kernel_mask, 
            strides=self.strides,
            padding='valid',
            data_format=self.data_format,
            dilation_rate=self.dilation_rate
        )

        # Apply convolutions to image
        img_output = K.conv2d(
            (images*masks), self.kernel, 
            strides=self.strides,
            padding='valid',
            data_format=self.data_format,
            dilation_rate=self.dilation_rate
        )        

        # Calculate the mask ratio on each pixel in the output mask
        mask_ratio = self.window_size / (mask_output + 1e-8)

        # Clip output to be between 0 and 1
        mask_output = K.clip(mask_output, 0, 1)

        # Remove ratio values where there are holes
        mask_ratio = mask_ratio * mask_output

        # Normalize iamge output
        img_output = img_output * mask_ratio

        # Apply bias only to the image (if chosen to do so)
        if self.use_bias:
            img_output = K.bias_add(
                img_output,
                self.bias,
                data_format=self.data_format)
        
        # Apply activations on the image
        if self.activation is not None:
            img_output = self.activation(img_output)
            
        return [img_output, mask_output]
    
    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            space = input_shape[0][1:-1]
            new_space = []
            for i in range(len(space)):
                new_dim = conv_output_length(
                    space[i],
                    self.kernel_size[i],
                    padding='same',
                    stride=self.strides[i],
                    dilation=self.dilation_rate[i])
                new_space.append(new_dim)
            new_shape = (input_shape[0][0],) + tuple(new_space) + (self.filters,)
            return [new_shape, new_shape]
        if self.data_format == 'channels_first':
            space = input_shape[2:]
            new_space = []
            for i in range(len(space)):
                new_dim = conv_output_length(
                    space[i],
                    self.kernel_size[i],
                    padding='same',
                    stride=self.strides[i],
                    dilation=self.dilation_rate[i])
                new_space.append(new_dim)
            new_shape = (input_shape[0], self.filters) + tuple(new_space)
            return [new_shape, new_shape]

## Reference: https://github.com/keras-team/keras/blob/7a39b6c62d43c25472b2c2476bd2a8983ae4f682/keras/utils/conv_utils.py#L85
def conv_output_length(input_length, filter_size,
                       padding, stride, dilation=1):
    if input_length is None:
        return None
    assert padding in {'same', 'valid', 'full', 'causal'}
    dilated_filter_size = (filter_size - 1) * dilation + 1
    if padding == 'same':
        output_length = input_length
    elif padding == 'valid':
        output_length = input_length - dilated_filter_size + 1
    elif padding == 'causal':
        output_length = input_length
    elif padding == 'full':
        output_length = input_length + dilated_filter_size - 1
    return (output_length + stride - 1) // stride


class InpaintingModel_256:
  '''
  Build UNET like model for image inpaining task.
  '''
  def prepare_model(self, input_size=(256, 256, 3)):
    input_image = keras.layers.Input(input_size)
    input_mask = keras.layers.Input(input_size, name='encoder_input')

    conv1, mask1, conv2, mask2 = self.__encoder_layer(32, input_image, input_mask, ['conv1', 'conv2']) # 3 -> 32

    conv3, mask3, conv4, mask4 = self.__encoder_layer(64, conv2, mask2, ['conv3', 'conv4']) # 32 -> 64
    conv5, mask5, conv6, mask6 = self.__encoder_layer(64, conv4, mask4, ['conv5', 'conv6']) # 64 -> 64
    conv7, mask7, conv8, mask8 = self.__encoder_layer(128, conv6, mask6, ['conv7', 'conv8']) # 64 -> 128

    conv9, mask9, conv10, mask10 = self.__encoder_layer(128, conv8, mask8, ['conv9', 'conv10']) # 128 -> 128
    conv11, mask11, conv12, mask12 = self.__encoder_layer(256, conv10, mask10, ['conv11', 'conv12']) # 128 -> 256
    conv13, mask13, conv14, mask14 = self.__encoder_layer(512, conv12, mask12, ['conv13', 'encoder_output']) # 256 -> 512
    # conv15, mask15, conv16, mask16 = self.__encoder_layer(512, conv14, mask14, ['conv15', 'encoder_output']) # 512 -> 512

    conv15, mask15, conv16, mask16 = self.__decoder_layer(512, 256, conv14, mask14, conv13, mask13, ['conv15', 'conv16'])
    conv17, mask17, conv18, mask18 = self.__decoder_layer(256, 128, conv16, mask16, conv11, mask11, ['conv17', 'conv18'])
    conv19, mask19, conv20, mask20 = self.__decoder_layer(128, 128, conv18, mask18, conv9, mask9, ['conv19', 'conv20'])
    conv21, mask21, conv22, mask22 = self.__decoder_layer(128, 64, conv20, mask20, conv7, mask7, ['conv21', 'conv22'])
    conv23, mask23, conv24, mask24 = self.__decoder_layer(64, 64, conv22, mask22, conv5, mask5, ['conv23', 'conv24'])
    conv25, mask25, conv26, mask26 = self.__decoder_layer(64, 32, conv24, mask24, conv3, mask3, ['conv25', 'conv26'])
    conv27, mask27, conv28, mask28 = self.__decoder_layer(32, 16, conv26, mask26, conv1, mask1, ['conv27', 'decoder_output'])

    # conv29, mask29, conv30, mask30 = self.__decoder_layer(64, 32, conv28, mask28, conv3, mask3, ['conv29', 'conv30'])
    # conv31, mask31, conv32, mask32 = self.__decoder_layer(32, 3, conv30, mask30, conv1, mask1, ['conv31', 'decoder_output'])

    outputs = keras.layers.Conv2D(3, (1, 1), activation='sigmoid', padding='same')(conv28)

    return keras.models.Model(inputs=[input_image, input_mask], outputs=[outputs])

    # return keras.models.Model(inputs=[input_image, input_mask], outputs=[conv16])

    
  def __encoder_layer(self, filters, in_layer, in_mask, names):
    conv1, mask1 = PConv2D(32, (3,3), strides=1, padding='same', name=names[0])([in_layer, in_mask])
    conv1 = keras.activations.relu(conv1)

    conv2, mask2 = PConv2D(32, (3,3), strides=2, padding='same', name=names[1])([conv1, mask1])
    conv2 = keras.layers.BatchNormalization(trainable=False)(conv2, training=True)
    conv2 = keras.activations.relu(conv2)

    return conv1, mask1, conv2, mask2

  def __decoder_layer(self, filter1, filter2, in_img, in_mask, share_img, share_mask, names):
    up_img = keras.layers.UpSampling2D(size=(2,2))(in_img)
    up_mask = keras.layers.UpSampling2D(size=(2,2))(in_mask)
    concat_img = keras.layers.Concatenate(axis=3)([share_img, up_img])
    concat_mask = keras.layers.Concatenate(axis=3)([share_mask, up_mask])

    conv1, mask1 = PConv2D(filter1, (3,3), padding='same', name=names[0])([concat_img, concat_mask])
    conv1 = keras.activations.relu(conv1)

    conv2, mask2 = PConv2D(filter2, (3,3), padding='same', name=names[1])([conv1, mask1])
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.activations.relu(conv2)

    return conv1, mask1, conv2, mask2

model_path = "/content/inpaitining_model/inpainting_model_best"

inpainting_model = InpaintingModel_256().prepare_model()
inpainting_model.load_weights(model_path)
inpainting_model.trainable = False

In [43]:
def inpaint_module(name, file_path, mask_path, save_dir):
    IMG_SIZE = 256

    mask_image = cv2.imread(file_path, cv2.IMREAD_COLOR)
    mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2RGB)
    og_h, og_w = mask_image.shape[:2]

    mask = cv2.imread(mask_path, cv2.IMREAD_COLOR)
    mask_copy = 255 - mask

    mask_image = cv2.bitwise_or(mask_copy, mask_image)
    # mask_copy = cv2.cvtColor(mask_copy, cv2.COLOR_BGR2GRAY)
    
    # mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    mask_image = cv2.resize(mask_image, (IMG_SIZE, IMG_SIZE), cv2.INTER_CUBIC)
    mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), cv2.INTER_CUBIC)

    mask_image = np.asarray(mask_image) / 255.0
    mask = np.asarray(mask) / 255.0

    mask_image = np.expand_dims(mask_image, 0)
    mask = np.expand_dims(mask, 0)

    # print(mask_image.shape, mask.shape)

    inputs = [mask_image, mask]
    impainted_image = inpainting_model.predict(inputs)[0]
    

    # pil_image = Image.fromarray(impainted_image)
    impainted_image = Image.fromarray(tf.cast(impainted_image* 255, tf.uint8).numpy() )

    # print(impainted_image.shape, type(impainted_image))

    impainted_image = impainted_image.resize((og_h, og_w),  Image.BICUBIC)

    name = "out_"+name
    save_path = os.path.join(save_dir, name)
    
    impainted_image.save(save_path)
    return save_path

clear_output()

# inpaint_module("006.png", "/content/static/uploads/download.png", "/content/static/uploads/masked_download.png", "/content/static/uploads")

In [44]:
# #@title FLASK APP
import base64

app = Flask(__name__)
run_with_ngrok(app)
app.config['UPLOAD_EXTENSIONS'] = ['.jpeg', '.png', '.jpg']
app.config['UPLOAD_PATH'] = r'/content/static/uploads'
app.config['OUTPUT_PATH'] = r"/content/static/OUTPUTS"


@app.route('/')
def index() :
    return render_template('index.html')

@app.route('/upload-image/<pathname>')
def uplaod_image(pathname) :
    path = request.path
    module_name = path.split('/')[-1]
    return render_template('upload.html', pathname=module_name)

@app.route('/upload-image/<pathname>', methods=['POST'])
def upload_files(pathname):
    
    path = request.path
    module_name = path.split('/')[-1]
    uploaded_file = request.files['file']
    file_name = uploaded_file.filename
    file_path = os.path.join(r"/content/static/uploads", file_name)
    # output_file = request.files['file']

    
    if file_name != '':
        file_ext = os.path.splitext(file_name)[1]
        if file_ext not in app.config['UPLOAD_EXTENSIONS'] :
            abort(400)

        uploaded_file.save(os.path.join(app.config['UPLOAD_PATH'], file_name))
        # output_file.save(os.path.join(app.config['OUTPUT_PATH'], filename))
        if pathname == 'inpaint' :
            image = request.form["payload"].split(',')[1] 
            img = Image.open(BytesIO(base64.b64decode(image)))
            rgb_im = img.convert('RGB')
            rgb_im.save(os.path.join(app.config['UPLOAD_PATH'], 'masked_'+file_name))   #saving file to directory

        # return filename
        # else :
        #   return "<p>404 Not found image. Please upload image</p>"
    
    print(module_name, file_path)
    if module_name == "super-resolution":
        if os.path.exists(file_path):
            _ = super_resolution_module(file_name, file_path, app.config['UPLOAD_PATH'])
        else:
            raise FileNotFoundError(f"File: {file_path} not present")
        
    elif module_name == "colorization":
        if os.path.exists(file_path):
            _ = colorisation_module(file_name, file_path, app.config['UPLOAD_PATH'])
        else:
            raise FileNotFoundError(f"File: {file_path} not present")
    
    elif module_name == "dehaze":
        if os.path.exists(file_path):
            _ = dehazing_module(file_name, file_path, app.config['UPLOAD_PATH'])
        else:
            raise FileNotFoundError(f"File: {file_path} not present")
        
    
    elif module_name == "inpaint":
        if os.path.exists(file_path):
            _ = inpaint_module(file_name, 
                               file_path, 
                               os.path.join(app.config['UPLOAD_PATH'], f'masked_{file_name}'), 
                               app.config['UPLOAD_PATH']
            )
        else:
            raise FileNotFoundError(f"File: {file_path} not present")
        pass
        

    return redirect(url_for('uploaded_file', 
                            pathname=pathname,
                            filename= 'masked_'+file_name if pathname == "inpaint" else file_name)
    )  #calling uploaded_file function
    
        
@app.route('/uploads/<filename>')
def send_file(filename):
    # print("SEND FILE")
    return send_from_directory(app.config['UPLOAD_PATH'], filename)

@app.route('/uploads/<filename>')
def send_output_file(filename):
    print("Found Rendering")
    return send_from_directory(r"/content/static/OUTPUTS", filename)


@app.route('/upload-image/<pathname>/<filename>')
def uploaded_file(pathname,filename):
    # filename = 'http://127.0.0.1:5000/uploads/' + filename
    return render_template('upload.html',pathname=pathname,filename=filename)

if __name__ == "__main__" :
    
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4fd07f09422d.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Apr/2021 15:30:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:30:32] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:30:33] "GET /static/images/sr.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:30:33] "GET /static/images/inpaint.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:30:33] "GET /static/images/dehaze.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:30:33] "GET /static/images/color-output.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:30:33] "GET /static/images/sr-output.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:30:33] "GET /static/images/color.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:30:34] "GET /static/images/In-paint_1.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:30:34] "GET /static/images/haze-output.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:30:36] "GET /upload-image/dehaze HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:30:36] "GET /static/js/script.js HTTP/1.1" 200 -
127.0.0.1 - - [

dehaze /content/static/uploads/2_images.jpg


...

dehaze /content/static/uploads/colo_2.jpg


127.0.0.1 - - [22/Apr/2021 15:31:57] "POST /upload-image/dehaze HTTP/1.1" 302 -


torch.Size([3, 291, 173])


127.0.0.1 - - [22/Apr/2021 15:31:57] "GET /upload-image/dehaze/colo_2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:31:58] "GET /static/uploads/out_colo_2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:31:58] "GET /uploads/colo_2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:32:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:32:38] "GET /upload-image/inpaint HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:32:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 15:32:46] "GET /upload-image/super-resolution HTTP/1.1" 200 -


super-resolution /content/static/uploads/56ac2-oldman.jpg


In [ ]:
# !zip -r files.zip /content/static /content/templates